# Loan Prediction
_Crystal Xu_
_____


In [2]:
import pandas as pd
import numpy as np 
from sklearn.compose import ColumnTransformer
import sklearn.preprocessing as pp
from sklearn.model_selection import train_test_split
import datetime
import matplotlib as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVR
import pickle

# 1. Data Import

In [8]:
train = pd.read_csv("Data for Cleaning & Modeling.csv")
train.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32
0,11.89%,54734.0,80364.0,"$25,000","$25,000","$19,080",36 months,B,B4,NaN,...,Feb-94,0.0,NaN,NaN,10.0,0.0,28854.0,52.10%,42.0,f
1,10.71%,55742.0,114426.0,"$7,000","$7,000",$673,36 months,B,B5,CNN,...,Oct-00,0.0,NaN,NaN,7.0,0.0,33623.0,76.70%,7.0,f
2,16.99%,57167.0,137225.0,"$25,000","$25,000","$24,725",36 months,D,D3,Web Programmer,...,Jun-00,0.0,41.0,NaN,10.0,0.0,19878.0,66.30%,17.0,f
3,13.11%,57245.0,138150.0,"$1,200","$1,200","$1,200",36 months,C,C2,city of beaumont texas,...,Jan-85,0.0,64.0,NaN,5.0,0.0,2584.0,40.40%,31.0,f
4,13.57%,57416.0,139635.0,"$10,800","$10,800","$10,692",36 months,C,C3,State Farm Insurance,...,Dec-96,1.0,58.0,NaN,14.0,0.0,3511.0,25.60%,40.0,f


# 2. Data Cleaning & Feature Engineering

Drop some columns based on its definition and contents

In [9]:
#Drop some columns that do not provide helpful information like ID 
#Drop some columns with low quality like X23 

remove_features = ['X2','X3','X10','X18',"X19","X23"]
keep_features = [col for col in list(train) if col not in remove_features]
train = train[keep_features]
train.head()

,X1,X4,X5,X6,X7,X8,X9,X11,X12,X13,...,X22,X24,X25,X26,X27,X28,X29,X30,X31,X32
0,11.89%,"$25,000","$25,000","$19,080",36 months,B,B4,< 1 year,RENT,85000.0,...,0.0,0.0,NaN,NaN,10.0,0.0,28854.0,52.10%,42.0,f
1,10.71%,"$7,000","$7,000",$673,36 months,B,B5,< 1 year,RENT,65000.0,...,0.0,0.0,NaN,NaN,7.0,0.0,33623.0,76.70%,7.0,f
2,16.99%,"$25,000","$25,000","$24,725",36 months,D,D3,1 year,RENT,70000.0,...,0.0,0.0,41.0,NaN,10.0,0.0,19878.0,66.30%,17.0,f
3,13.11%,"$1,200","$1,200","$1,200",36 months,C,C2,10+ years,OWN,54000.0,...,0.0,0.0,64.0,NaN,5.0,0.0,2584.0,40.40%,31.0,f
4,13.57%,"$10,800","$10,800","$10,692",36 months,C,C3,6 years,RENT,32000.0,...,0.0,1.0,58.0,NaN,14.0,0.0,3511.0,25.60%,40.0,f


Check NaN

In [10]:
(train.isna().sum()/len(train)).sort_values(ascending=False)

X26    0.872112
X16    0.691100
X25    0.547005
X12    0.153402
X8     0.153175
X9     0.153175
X13    0.152570
X1     0.152525
X11    0.043845
X30    0.000668
X22    0.000003
X31    0.000003
X29    0.000003
X28    0.000003
X27    0.000003
X24    0.000003
X17    0.000003
X21    0.000003
X20    0.000003
X4     0.000003
X15    0.000003
X14    0.000003
X7     0.000003
X6     0.000003
X5     0.000003
X32    0.000003
dtype: float64

In [11]:
# Drop X26 X25 X16 which have too much NaN
remove_features = ['X26',"X25","X16"]
keep_features = [col for col in list(train) if col not in remove_features]
train = train[keep_features]
train.head()

,X1,X4,X5,X6,X7,X8,X9,X11,X12,X13,...,X20,X21,X22,X24,X27,X28,X29,X30,X31,X32
0,11.89%,"$25,000","$25,000","$19,080",36 months,B,B4,< 1 year,RENT,85000.0,...,CA,19.48,0.0,0.0,10.0,0.0,28854.0,52.10%,42.0,f
1,10.71%,"$7,000","$7,000",$673,36 months,B,B5,< 1 year,RENT,65000.0,...,NY,14.29,0.0,0.0,7.0,0.0,33623.0,76.70%,7.0,f
2,16.99%,"$25,000","$25,000","$24,725",36 months,D,D3,1 year,RENT,70000.0,...,NY,10.50,0.0,0.0,10.0,0.0,19878.0,66.30%,17.0,f
3,13.11%,"$1,200","$1,200","$1,200",36 months,C,C2,10+ years,OWN,54000.0,...,TX,5.47,0.0,0.0,5.0,0.0,2584.0,40.40%,31.0,f
4,13.57%,"$10,800","$10,800","$10,692",36 months,C,C3,6 years,RENT,32000.0,...,CT,11.63,0.0,1.0,14.0,0.0,3511.0,25.60%,40.0,f


In [12]:
# drop the row that X1 is NA
train.dropna(subset=["X1"], inplace=True)
(train.isna().sum()/len(train)).sort_values(ascending=False)

X12    0.153279
X8     0.153005
X9     0.153005
X13    0.152665
X11    0.043644
X30    0.000664
X15    0.000003
X21    0.000003
X31    0.000003
X29    0.000003
X28    0.000003
X27    0.000003
X24    0.000003
X22    0.000003
X32    0.000003
X20    0.000003
X17    0.000003
X4     0.000003
X14    0.000003
X7     0.000003
X6     0.000003
X5     0.000003
X1     0.000000
dtype: float64

In [13]:
#check the case that X22 is NaN
train[train["X22"].isna()==True]
#these several lines are all NaN -> eliminate
train = train[train["X22"].isna()==False]
(train.isna().sum()).sort_values(ascending=False).head(10)

X12    51959
X8     51866
X9     51866
X13    51751
X11    14794
X30      224
X1         0
X21        0
X31        0
X29        0
dtype: int64

check X12

In [14]:
train["X12"].value_counts()

MORTGAGE    145958
RENT        115958
OWN          24976
OTHER          107
NONE            30
ANY              1
Name: X12, dtype: int64

In [15]:
# change none and any to toher 
# fill na with other 
train["X12"].fillna("OTHER",inplace = True)
train["X12"]= train["X12"].replace(["ANY","NONE"], "OTHER")
train["X12"].value_counts()

MORTGAGE    145958
RENT        115958
OTHER        52097
OWN          24976
Name: X12, dtype: int64

Explore X8 X9

In [16]:
# X9 catches most of the information in X8, X8 is redundant -> Drop X8
train.drop("X8", inplace=True, axis=1)
train["X9"].value_counts().head(10)

B3    20352
B4    19137
B2    16767
C1    16342
C2    16310
B5    15521
C3    15425
C4    14646
B1    14344
C5    13723
Name: X9, dtype: int64

In [17]:
# "X9" has 61269 na, which is more than any other categories. So we cannot change the NA to any category -> Drop NA
train.dropna(subset=["X9"], inplace=True)

check X11

In [18]:
train["X11"].value_counts()

10+ years    91870
2 years      25550
3 years      22613
< 1 year     22029
5 years      19514
1 year       18132
4 years      17220
6 years      16576
7 years      16447
8 years      13773
9 years      10948
Name: X11, dtype: int64

In [19]:
# X11 has 12451 NA, fill it with the mode
train["X11"] = train["X11"].fillna(train['X11'].mode()[0])

Typecast, clean the data

In [20]:
train["X1"] = train["X1"].str.strip("%").astype(float)/100
train["X4"] = train["X4"].str.strip("$").str.replace(",","").astype(float)
train["X5"] = train["X5"].str.strip("$").str.replace(",","").astype(float)
train["X6"] = train["X6"].str.strip("$").str.replace(",","").astype(float)
train["X7"] = train["X7"].str.strip("months").astype(float)
train["X30"] = train["X30"].str.strip("%").astype(float)/100

In [21]:
train.head(10)

,X1,X4,X5,X6,X7,X9,X11,X12,X13,X14,...,X20,X21,X22,X24,X27,X28,X29,X30,X31,X32
0,0.1189,25000.0,25000.0,19080.0,36.0,B4,< 1 year,RENT,85000.0,VERIFIED - income,...,CA,19.48,0.0,0.0,10.0,0.0,28854.0,0.521,42.0,f
1,0.1071,7000.0,7000.0,673.0,36.0,B5,< 1 year,RENT,65000.0,not verified,...,NY,14.29,0.0,0.0,7.0,0.0,33623.0,0.767,7.0,f
2,0.1699,25000.0,25000.0,24725.0,36.0,D3,1 year,RENT,70000.0,VERIFIED - income,...,NY,10.50,0.0,0.0,10.0,0.0,19878.0,0.663,17.0,f
3,0.1311,1200.0,1200.0,1200.0,36.0,C2,10+ years,OWN,54000.0,not verified,...,TX,5.47,0.0,0.0,5.0,0.0,2584.0,0.404,31.0,f
4,0.1357,10800.0,10800.0,10692.0,36.0,C3,6 years,RENT,32000.0,not verified,...,CT,11.63,0.0,1.0,14.0,0.0,3511.0,0.256,40.0,f
5,0.1905,7200.0,7200.0,7200.0,36.0,D4,9 years,RENT,58000.0,VERIFIED - income source,...,RI,2.05,0.0,0.0,6.0,0.0,3874.0,0.901,25.0,f
6,0.1008,7500.0,5025.0,557.0,36.0,B3,3 years,RENT,85000.0,not verified,...,CA,8.10,0.0,1.0,3.0,0.0,33667.0,0.732,11.0,f
7,0.1426,3000.0,3000.0,2988.0,36.0,C5,3 years,MORTGAGE,80800.0,not verified,...,TX,14.97,1.0,0.0,13.0,0.0,4740.0,0.395,23.0,f
8,0.0788,4000.0,4000.0,3900.0,36.0,A5,< 1 year,MORTGAGE,148000.0,not verified,...,TX,16.98,0.0,0.0,11.0,0.0,50807.0,0.510,19.0,f
9,0.1496,5600.0,5600.0,5525.0,36.0,D2,1 year,RENT,45000.0,not verified,...,MD,4.00,0.0,0.0,5.0,1.0,3839.0,0.768,9.0,f


Fill X30 and X13

In [22]:
train["X13"] = train["X13"].fillna(train["X13"].mean())
train["X30"] = train["X30"].fillna(train["X30"].mean())

In [23]:
(train.isna().sum()).sort_values(ascending=False)

X1     0
X4     0
X31    0
X30    0
X29    0
X28    0
X27    0
X24    0
X22    0
X21    0
X20    0
X17    0
X15    0
X14    0
X13    0
X12    0
X11    0
X9     0
X7     0
X6     0
X5     0
X32    0
dtype: int64

Explore X15

In [24]:
# extract the month and date 
train["X15_month"] = pd.to_datetime(train["X15"],dayfirst=True,format="%b-%d").dt.month
train["X15"] = pd.to_datetime(train["X15"],dayfirst=True,format="%b-%d").dt.day

In [25]:
train.head(10)

,X1,X4,X5,X6,X7,X9,X11,X12,X13,X14,...,X21,X22,X24,X27,X28,X29,X30,X31,X32,X15_month
0,0.1189,25000.0,25000.0,19080.0,36.0,B4,< 1 year,RENT,85000.0,VERIFIED - income,...,19.48,0.0,0.0,10.0,0.0,28854.0,0.521,42.0,f,8
1,0.1071,7000.0,7000.0,673.0,36.0,B5,< 1 year,RENT,65000.0,not verified,...,14.29,0.0,0.0,7.0,0.0,33623.0,0.767,7.0,f,5
2,0.1699,25000.0,25000.0,24725.0,36.0,D3,1 year,RENT,70000.0,VERIFIED - income,...,10.50,0.0,0.0,10.0,0.0,19878.0,0.663,17.0,f,8
3,0.1311,1200.0,1200.0,1200.0,36.0,C2,10+ years,OWN,54000.0,not verified,...,5.47,0.0,0.0,5.0,0.0,2584.0,0.404,31.0,f,3
4,0.1357,10800.0,10800.0,10692.0,36.0,C3,6 years,RENT,32000.0,not verified,...,11.63,0.0,1.0,14.0,0.0,3511.0,0.256,40.0,f,11
5,0.1905,7200.0,7200.0,7200.0,36.0,D4,9 years,RENT,58000.0,VERIFIED - income source,...,2.05,0.0,0.0,6.0,0.0,3874.0,0.901,25.0,f,8
6,0.1008,7500.0,5025.0,557.0,36.0,B3,3 years,RENT,85000.0,not verified,...,8.10,0.0,1.0,3.0,0.0,33667.0,0.732,11.0,f,4
7,0.1426,3000.0,3000.0,2988.0,36.0,C5,3 years,MORTGAGE,80800.0,not verified,...,14.97,1.0,0.0,13.0,0.0,4740.0,0.395,23.0,f,9
8,0.0788,4000.0,4000.0,3900.0,36.0,A5,< 1 year,MORTGAGE,148000.0,not verified,...,16.98,0.0,0.0,11.0,0.0,50807.0,0.510,19.0,f,2
9,0.1496,5600.0,5600.0,5525.0,36.0,D2,1 year,RENT,45000.0,not verified,...,4.00,0.0,0.0,5.0,1.0,3839.0,0.768,9.0,f,2


In [26]:
train.dtypes

X1           float64
X4           float64
X5           float64
X6           float64
X7           float64
X9            object
X11           object
X12           object
X13          float64
X14           object
X15            int64
X17           object
X20           object
X21          float64
X22          float64
X24          float64
X27          float64
X28          float64
X29          float64
X30          float64
X31          float64
X32           object
X15_month      int64
dtype: object

In [27]:
train.shape

(287123, 23)

# 3. Modele Fitting

In [28]:
y = train["X1"]
x = train.drop("X1",axis=1)

In [29]:
# standardize, and change the categories to dummy variables
 
num_feat = ['X4','X5','X6','X7','X13','X15','X21','X22','X24','X27','X28','X29','X30','X31','X15_month']
cat_feat = ['X9','X11','X12','X14','X17','X20','X32']
preproc = ColumnTransformer(transformers=[('num', pp.StandardScaler(), num_feat), ('cat', pp.OneHotEncoder(handle_unknown = 'ignore'), cat_feat)])
preproc.fit(x)
x_trans = preproc.transform(x)
X_train, X_test, y_train, y_test = train_test_split(x_trans, y, test_size=0.3)

In [30]:
x_trans.shape

(287123, 134)

### linearRegression 

In [57]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_lr = lr.predict(X_test)
y_base = lr.predict(X_train)
print("RMSE Train = {}".format(np.sqrt(np.mean((y_base - y_train)**2))))
print("RMSE = {}".format(np.sqrt(np.mean((y_lr - y_test)**2))))
print("Mean squared error: {}" .format( mean_squared_error(y_test, y_lr)))
print("R square: {}" .format(r2_score(y_test, y_lr)))

#leave cross validation
MSE_CV_scores_lr = - cross_val_score(lr, X_train, y_train, cv=10, 
                                  scoring='neg_mean_squared_error', 
                                  n_jobs=-1) 

RMSE_CV = (MSE_CV_scores_lr.mean())**(1/2)

print('CV RMSE: {}'.format(RMSE_CV))

RMSE Train = 0.007769731311841345
RMSE = 0.007853335407347837
Mean squared error: 6.167487702030332e-05
R square: 0.9679864095460783
CV RMSE: 0.0077769684198929715


### Decision Tree Regression

In [58]:
dt = DecisionTreeRegressor(random_state=1)
dt.fit(X_train,y_train)
y_dt = dt.predict(X_test)
y_base = dt.predict(X_train)
print("RMSE Train = {}".format(np.sqrt(np.mean((y_base - y_train)**2))))
print("RMSE = {}".format(np.sqrt(np.mean((y_dt - y_test)**2))))
print("Mean squared error: {}" .format( mean_squared_error(y_test, y_dt)))
print("R square: {}" .format(r2_score(y_test, y_dt)))
MSE_CV_scores_dt = - cross_val_score(dt, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1) 
RMSE_CV = (MSE_CV_scores_dt.mean())**(1/2)
print('CV RMSE: {}'.format(RMSE_CV))

RMSE Train = 1.2466310998301274e-15
RMSE = 0.002564578555399632
Mean squared error: 6.577063166815663e-06
R square: 0.9965860425381522
CV RMSE: 0.002857572368877209


The Train RMSE is a lot lower than Test RMSE -> over fitting </br>
Cannot reduce the max_depth, so increase the min_samples_split

In [102]:
parameters_refine = {
    'max_depth': [1,None,5,10], 
    'min_samples_split':[2,5,10,50],
    'min_samples_leaf':[1,5,10],
}
gscv_r = GridSearchCV(DecisionTreeRegressor(), parameters_refine, cv=5)
gscv_r.fit(X_train, y_train)
gscv_r.best_params_

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 50}

In [59]:
dt = DecisionTreeRegressor(max_depth= None, min_samples_leaf = 1,min_samples_split = 50, random_state=1)
dt.fit(X_train,y_train)
y_dt = dt.predict(X_test)
y_base = dt.predict(X_train)
print("RMSE Train = {}".format(np.sqrt(np.mean((y_base - y_train)**2))))
print("RMSE = {}".format(np.sqrt(np.mean((y_dt - y_test)**2))))
print("Mean squared error: {}" .format( mean_squared_error(y_test, y_dt)))
print("R square: {}" .format(r2_score(y_test, y_dt)))

#leave one out cross validation
MSE_CV_scores_dt = - cross_val_score(dt, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1) 
RMSE_CV = (MSE_CV_scores_dt.mean())**(1/2)
print('CV RMSE: {}'.format(RMSE_CV))

RMSE Train = 0.0017750631148638534
RMSE = 0.0023062162265672496
Mean squared error: 5.318633283682097e-06
R square: 0.997239255983237
CV RMSE: 0.0024143704870412948


### RandomForestRegressor

In [55]:
rf = RandomForestRegressor(random_state=1)
rf.fit(X_train,y_train)
y_rf = rf.predict(X_test)
y_base = rf.predict(X_train)
print("RMSE Train = {}".format(np.sqrt(np.mean((y_base - y_train)**2))))
print("RMSE = {}".format(np.sqrt(np.mean((y_rf - y_test)**2))))
print("Mean squared error: {}" .format( mean_squared_error(y_test, y_rf)))
print("R square: {}" .format(r2_score(y_test, y_rf)))

RMSE Train = 0.0007658930900191411
RMSE = 0.002081552814416553
Mean squared error: 4.3328621192054605e-06
R square: 0.9977509404140054


In [56]:
MSE_CV_scores_rf = - cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1) 
RMSE_CV = (MSE_CV_scores_rf.mean())**(1/2)
print('CV RMSE: {}'.format(RMSE_CV))

CV RMSE: 0.0021324293250267307


### SVM Regression

In [70]:
sv = SVR(C=1)
sv.fit(X_train,y_train)
y_svm = sv.predict(X_test)
y_base = sv.predict(X_train)
print("RMSE Train = {}".format(np.sqrt(np.mean((y_base - y_train)**2))))
print("RMSE = {}".format(np.sqrt(np.mean((y_svm - y_test)**2))))
print("Mean squared error: {}" .format( mean_squared_error(y_test, y_svm)))
print("R square: {}" .format(r2_score(y_test, y_svm)))
MSE_CV_scores_svm = - cross_val_score(sv, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1) 
RMSE_CV = (MSE_CV_scores_svm.mean())**(1/2)
print('CV RMSE: {}'.format(RMSE_CV))

RMSE Train = 0.04701771637146462
RMSE = 0.047089141896995264
Mean squared error: 0.002217387284595388
R square: -0.15097965065097174
CV RMSE: 0.04698282686408149


### Comparison

In [73]:
result = pd.DataFrame(columns=('Model Type', 'MSE','R Square','CV_mean'))
result.loc[0]= ["Linear Regression", mean_squared_error(y_test, y_lr), r2_score(y_test, y_lr),MSE_CV_scores_lr.mean()]
result.loc[1]= ["Decision Tree Regression", mean_squared_error(y_test, y_dt), r2_score(y_test, y_dt),MSE_CV_scores_dt.mean()]
result.loc[2]= ["Random Forest Regression", mean_squared_error(y_test, y_rf), r2_score(y_test, y_rf),MSE_CV_scores_rf.mean()]
result.loc[3]= ["SVM Regression", mean_squared_error(y_test, y_svm), r2_score(y_test, y_svm),MSE_CV_scores_svm.mean()]
result

,Model Type,MSE,R Square,CV_mean
0,Linear Regression,0.000062,0.967986,0.000060
1,Decision Tree Regression,0.000005,0.997239,0.000006
2,Random Forest Regression,0.000004,0.997751,0.000005
3,SVM Regression,0.002217,-0.150980,0.002207


Comparison </br>

Performance measured by RMSE
- Random Forest Regression: Although the model has best MSE, R square and cv score, training cost is high. 
- Linear Regression: MSE is relatively high 
- Decision Tree Regression: The performance is good and the training cost is lower than random forest. If the data set gets larger, decision tree should be a great option. 
- SVM Regression: R square is negative, the model is bad

Choose random forest because the performance is better. 

In [75]:
filename = "finalized_model.sav"
pickle.dump(rf, open(filename, 'wb'))

# 4. Test Data

In [31]:
test = pd.read_csv("Holdout for Testing.csv")
test.drop("X1",inplace=True,axis=1)

In [32]:
(test.isnull().sum()/len(test)).sort_values(ascending=False).head(10)

X16    0.999812
X26    0.827013
X25    0.483800
X10    0.054925
X11    0.054775
X30    0.000375
X21    0.000000
X22    0.000000
X23    0.000000
X24    0.000000
dtype: float64

In [33]:
remove_features = ['X2','X3','X10','X18',"X19","X23"]
keep_features = [col for col in list(test) if col not in remove_features]
test = test[keep_features]
remove_features = ['X26',"X25","X16"]
keep_features = [col for col in list(test) if col not in remove_features]
test = test[keep_features]
test= test[test["X22"].isna()==False]
test["X12"].fillna("OTHER",inplace = True)
test["X12"]= test["X12"].replace(["ANY","NONE"], "OTHER")
test.drop("X8", inplace=True, axis=1)
test.dropna(subset=["X9"], inplace=True)
test["X11"] = test["X11"].fillna(test['X11'].mode()[0])
#test["X1"] = test["X1"].str.strip("%").astype(float)/100
test["X4"] = test["X4"].str.strip("$").str.replace(",","").astype(float)
test["X5"] = test["X5"].str.strip("$").str.replace(",","").astype(float)
test["X6"] = test["X6"].str.strip("$").str.replace(",","").astype(float)
test["X7"] = test["X7"].str.strip("months").astype(float)
test["X30"] = test["X30"].str.strip("%").astype(float)/100
test["X13"] = test["X13"].fillna(test["X13"].mean())
test["X30"] = test["X30"].fillna(test["X30"].mean())
test["X15_month"] = pd.to_datetime(test["X15"],dayfirst=True,format="%d-%b").dt.month
test["X15"] = pd.to_datetime(test["X15"],dayfirst=True,format="%d-%b").dt.day

In [34]:
(test.isnull().sum()/len(test)).sort_values(ascending=False).head(10)

X4     0.0
X5     0.0
X32    0.0
X31    0.0
X30    0.0
X29    0.0
X28    0.0
X27    0.0
X24    0.0
X22    0.0
dtype: float64

In [35]:
test_trans = preproc.transform(test)
test_trans.shape

(80000, 134)

In [37]:
filename = "finalized_model.sav"
loaded_model = pickle.load(open(filename, 'rb'))

In [38]:
y_pred = loaded_model.predict(test_trans)
y_pred.shape

(80000,)

In [39]:
df = pd.DataFrame({"results": y_pred})
df.to_csv("Results.csv", header = 0, index =0)
df.head(10)

,results
0,0.156100
1,0.060300
2,0.142127
3,0.165900
4,0.096700
5,0.119000
6,0.146400
7,0.109900
8,0.153100
9,0.066200
